In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import datetime
from scipy import stats
from scipy.stats import linregress
from config import weather_api_key
#from config import g_key
from citipy import citipy
import csv as atlantic_working

In [2]:
# Import API key
from config import weather_api_key

In [3]:
# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

In [4]:
awdf = pd.read_csv('atlantic macro.csv')

In [5]:
awdf

,ID,Name,Date,Time,Event,Status,Latitude,Longitude,Maximum Wind,Minimum PreLow Intensity Subtropical Cycloneure,...,Low Wind SW,Low Wind NW,Moderate Wind NE,Moderate Wind SE,Moderate Wind SW,Moderate Wind NW,High Wind NE,High Wind SE,High Wind SW,High Wind NW
0,AL011851,UNNAMED,18510625,0,,Hurricane,28.0N,94.8W,80,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
1,AL011851,UNNAMED,18510625,600,,Hurricane,28.0N,95.4W,80,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
2,AL011851,UNNAMED,18510625,1200,,Hurricane,28.0N,96.0W,80,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
3,AL011851,UNNAMED,18510625,1800,,Hurricane,28.1N,96.5W,80,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
4,AL011851,UNNAMED,18510625,2100,Landfall,Hurricane,28.2N,96.8W,80,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49100,AL122015,KATE,20151112,1200,,Extratropical Cyclone,41.3N,50.4W,55,981,...,180,120,120,120,60,0,0,0,0,0
49101,AL122015,KATE,20151112,1800,,Extratropical Cyclone,41.9N,49.9W,55,983,...,180,120,120,120,60,0,0,0,0,0
49102,AL122015,KATE,20151113,0,,Extratropical Cyclone,41.5N,49.2W,50,985,...,200,220,120,120,60,0,0,0,0,0
49103,AL122015,KATE,20151113,600,,Extratropical Cyclone,40.8N,47.5W,45,985,...,180,220,0,0,0,0,0,0,0,0


In [6]:
df1 = awdf.copy()
df1

,ID,Name,Date,Time,Event,Status,Latitude,Longitude,Maximum Wind,Minimum PreLow Intensity Subtropical Cycloneure,...,Low Wind SW,Low Wind NW,Moderate Wind NE,Moderate Wind SE,Moderate Wind SW,Moderate Wind NW,High Wind NE,High Wind SE,High Wind SW,High Wind NW
0,AL011851,UNNAMED,18510625,0,,Hurricane,28.0N,94.8W,80,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
1,AL011851,UNNAMED,18510625,600,,Hurricane,28.0N,95.4W,80,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
2,AL011851,UNNAMED,18510625,1200,,Hurricane,28.0N,96.0W,80,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
3,AL011851,UNNAMED,18510625,1800,,Hurricane,28.1N,96.5W,80,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
4,AL011851,UNNAMED,18510625,2100,Landfall,Hurricane,28.2N,96.8W,80,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49100,AL122015,KATE,20151112,1200,,Extratropical Cyclone,41.3N,50.4W,55,981,...,180,120,120,120,60,0,0,0,0,0
49101,AL122015,KATE,20151112,1800,,Extratropical Cyclone,41.9N,49.9W,55,983,...,180,120,120,120,60,0,0,0,0,0
49102,AL122015,KATE,20151113,0,,Extratropical Cyclone,41.5N,49.2W,50,985,...,200,220,120,120,60,0,0,0,0,0
49103,AL122015,KATE,20151113,600,,Extratropical Cyclone,40.8N,47.5W,45,985,...,180,220,0,0,0,0,0,0,0,0


In [7]:
lat=df1["Latitude"].to_string()
lat = df1.loc[:, "Latitude"]
lat = lat.str.replace('N', "")
df1.loc[:,'Latitude'] = lat.astype('float')

In [8]:
lng=df1["Longitude"].to_string()
lng = df1.loc[:, "Longitude"]
lng = lng.str.replace('W|E', "")
df1.loc[:,'Longitude'] = lng.astype('float') * -1

#print(lng)

C:\Users\George\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  This is separate from the ipykernel package so we can avoid doing imports until


In [23]:
df1

,ID,Name,Date,Time,Event,Status,Latitude,Longitude,Maximum Wind,Minimum PreLow Intensity Subtropical Cycloneure,...,Low Wind SW,Low Wind NW,Moderate Wind NE,Moderate Wind SE,Moderate Wind SW,Moderate Wind NW,High Wind NE,High Wind SE,High Wind SW,High Wind NW
0,AL011851,UNNAMED,18510625,0,,Hurricane,28.0,-94.8,80,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
1,AL011851,UNNAMED,18510625,600,,Hurricane,28.0,-95.4,80,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
2,AL011851,UNNAMED,18510625,1200,,Hurricane,28.0,-96.0,80,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
3,AL011851,UNNAMED,18510625,1800,,Hurricane,28.1,-96.5,80,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
4,AL011851,UNNAMED,18510625,2100,Landfall,Hurricane,28.2,-96.8,80,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49100,AL122015,KATE,20151112,1200,,Extratropical Cyclone,41.3,-50.4,55,981,...,180,120,120,120,60,0,0,0,0,0
49101,AL122015,KATE,20151112,1800,,Extratropical Cyclone,41.9,-49.9,55,983,...,180,120,120,120,60,0,0,0,0,0
49102,AL122015,KATE,20151113,0,,Extratropical Cyclone,41.5,-49.2,50,985,...,200,220,120,120,60,0,0,0,0,0
49103,AL122015,KATE,20151113,600,,Extratropical Cyclone,40.8,-47.5,45,985,...,180,220,0,0,0,0,0,0,0,0


In [ ]:
# #set "city center" for Houston
# Houston_lat = 29.756097853207656
# Houston_lng = -95.3669907972107

# #calculate greater Houston area by expanding lat and lng
# lat = np.random.uniform(Houston_lat-1, Houston_lat+2, size=6500)
# lng = np.random.uniform(Houston_lng-5, Houston_lng+2, size=6500)

In [24]:
filtered_lat_lng = df1[(df1["Latitude"]<=29) & (df1["Latitude"]>=18) & (df1["Longitude"]>=-100) & (df1["Longitude"]<=-83)]
filtered_lat_lng

,ID,Name,Date,Time,Event,Status,Latitude,Longitude,Maximum Wind,Minimum PreLow Intensity Subtropical Cycloneure,...,Low Wind SW,Low Wind NW,Moderate Wind NE,Moderate Wind SE,Moderate Wind SW,Moderate Wind NW,High Wind NE,High Wind SE,High Wind SW,High Wind NW
0,AL011851,UNNAMED,18510625,0,,Hurricane,28.0,-94.8,80,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
1,AL011851,UNNAMED,18510625,600,,Hurricane,28.0,-95.4,80,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
2,AL011851,UNNAMED,18510625,1200,,Hurricane,28.0,-96.0,80,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
3,AL011851,UNNAMED,18510625,1800,,Hurricane,28.1,-96.5,80,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
4,AL011851,UNNAMED,18510625,2100,Landfall,Hurricane,28.2,-96.8,80,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48794,AL022015,BILL,20150616,0,,Tropical Storm,27.0,-94.3,45,1005,...,0,0,0,0,0,0,0,0,0,0
48795,AL022015,BILL,20150616,600,,Tropical Storm,27.6,-95.2,45,1001,...,0,0,0,0,0,0,0,0,0,0
48796,AL022015,BILL,20150616,1200,,Tropical Storm,28.0,-96.0,50,997,...,0,30,20,20,0,0,0,0,0,0
48797,AL022015,BILL,20150616,1645,Landfall,Tropical Storm,28.2,-96.7,50,997,...,0,30,20,20,0,0,0,0,0,0


In [25]:
filt_df=filtered_lat_lng.rename(columns={"Minimum PreLow Intensity Subtropical Cycloneure":"Minimum Pressure"})

In [26]:
filt_df2 = filt_df[(filt_df["Minimum Pressure"]>=0)]
filt_df2

,ID,Name,Date,Time,Event,Status,Latitude,Longitude,Maximum Wind,Minimum Pressure,...,Low Wind SW,Low Wind NW,Moderate Wind NE,Moderate Wind SE,Moderate Wind SW,Moderate Wind NW,High Wind NE,High Wind SE,High Wind SW,High Wind NW
1981,AL011871,UNNAMED,18710604,600,,Tropical Storm,28.9,-95.0,50,999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
3825,AL021880,UNNAMED,18800813,0,,Hurricane,25.7,-96.9,130,931,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
3826,AL021880,UNNAMED,18800813,100,Landfall,Hurricane,25.8,-97.0,130,931,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
3827,AL021880,UNNAMED,18800813,600,,Hurricane,26.1,-97.6,110,943,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
4275,AL021882,UNNAMED,18820907,0,,Hurricane,22.6,-85.8,80,981,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48794,AL022015,BILL,20150616,0,,Tropical Storm,27.0,-94.3,45,1005,...,0,0,0,0,0,0,0,0,0,0
48795,AL022015,BILL,20150616,600,,Tropical Storm,27.6,-95.2,45,1001,...,0,0,0,0,0,0,0,0,0,0
48796,AL022015,BILL,20150616,1200,,Tropical Storm,28.0,-96.0,50,997,...,0,30,20,20,0,0,0,0,0,0
48797,AL022015,BILL,20150616,1645,Landfall,Tropical Storm,28.2,-96.7,50,997,...,0,30,20,20,0,0,0,0,0,0


In [ ]:
#turn cities into df
df_city = pd.DataFrame({
    "City":city,
    "lat":lat,
    "lng":lng,
})